In [ ]:
!pip install langchain-openai langchain-community huggingface_hub PyPDF2 langchain-huggingface faiss-cpu langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00


In [ ]:
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from huggingface_hub import login
from langchain_groq import ChatGroq
import os

In [ ]:
load_dotenv()
login(os.getenv("HUGGINGFACEHUB_API_TOKEN"))

In [ ]:
from google.colab import files
uploaded = files.upload()

TypeError: 'NoneType' object is not subscriptable

In [ ]:
pdf_paths = [f"/content/{filename}" for filename in uploaded.keys()]
print(f"selected PDFs: {pdf_paths}")

In [ ]:
def get_pdf_text(pdf_docs):
  '''
  loop over all the pdf files and concatenate
  the content in a single string
  '''
  text = ""
  for pdf in pdf_paths:
    pdf_reader = PdfReader(pdf)
    for page in pdf_reader.pages:
      page_text = page.extract_text()
      if page_text:
        text += page_text
    return text


In [ ]:
raw_text = get_pdf_text(pdf_paths)
print("✅ Extracted text length:", len(raw_text))

In [ ]:
def get_text_chunks(raw_text):
  '''
  takes a single string of text and returns a list
  of text strings that can be fed to vector database
  '''
  text_splitter = CharacterTextSplitter(
      separator="\n",
      chunk_size=1000,
      chunk_overlap=200,
      length_function=len
  )
  chunks = text_splitter.split_text(raw_text)
  return chunks


In [ ]:
text_chunks = get_text_chunks(raw_text)
print("✅ Number of text chunks:", len(text_chunks))

In [ ]:
def get_vectorstore(text_chunks):
  '''
  creat a FAISS vectorstore from the text chunks using embeddings
  '''
  embeddings = HuggingFaceEmbeddings(
      model_name="hkunlp/instructor-xl",
      model_kwargs={"device": "cpu"}
  )
  vectorstore = FAISS.from_texts(
      texts=text_chunks,
      embedding=embeddings
  )
  return vectorstore


In [ ]:
vectorstore = get_vectorstore(text_chunks)
print("✅ vectorstore created")

In [ ]:
def get_conversation_chain(vectorstore):
    '''
    Creates a conversational retrieval chain using OpenAI's mode
    '''
   #llm = ChatOpenAI(
     # model_name= "gpt-4o-mini",
     # temperature=0.7,
     # request_timeout=30
    llm = ChatGroq(
         model_name="llama-3.3-70b-versatile",
         temperature=0.7,
         request_timeout=30,
         api_key="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"
    )
    memory = ConversationBufferMemory(
        memory_key='chat_history',
        return_messages=True
    )
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain


In [ ]:
conversation = get_conversation_chain(vectorstore)

In [ ]:
def chat_with_pdf(question):
    response = conversation({'question': question})
    for i, msg in enumerate(response['chat_history']):
      sender = "Query````````````1                                        " if i % 2 == 0 else "Timeline"
      print(f"{sender}: {msg.content}\n")

In [ ]:
chat_with_pdf("Events occured in 1985?")
chat_with_pdf("Timeline of Europian History?")
chat_with_pdf("What happened in 1938?")